In [ ]:
# 基于iwencai flag的超短线策略: 计算flag期望及分布

# 统计某段时间各flag的期望收益
# 轮训iwencai_flag_history以及ths_kday的数据并统计，每个flag出现的情况下：
# 1. 第二天的涨幅
# 2. 后五天的涨幅
# 3. 排除涨停的情况下第二天的涨幅
# 将所有收益纪录保存于
# iwencai_flag_stats_<xxxx>.json (xxxx是年份) 格式为 [{"flag":"...", "oneday":[...], "onedayf":[...], "fiveday":[...]}]
# 统计一日期望收益并存于： 
# iwencai_flag_performance_onedayf_<xxxx>.json 格式为 {<flag>: {"avr":..., "count":...}}

from opentrader.core.ticker import *
from opentrader.common.db import db_ot
from opentrader.common.utils import d2dt
from opentrader.agents.ths.api import symbol_convert3
from pymongo.errors import *
from datetime import datetime, timedelta

timetag = "2008_2014"
start_date = gen_time('2008-01-01 00:00:00').date()
end_date = gen_time('2014-12-31 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)

In [ ]:
# 将k线数据缓存在内存中
kday_cache = {} # {'...symbol...':[{}, {}, {}]}
cur_year = 0
for date in dates:
    if date.year != cur_year:
        print(date.year)
        cur_year = date.year
    else:
        print('.', end='')
    result = db_ot.ths_kday.find({'date':d2dt(date)})
    for each in result:
        if each['symbol'] not in kday_cache:
            kday_cache[each['symbol']] = []
        kday_cache[each['symbol']].append(each)

In [ ]:
# 从k线缓存获取数据的工具函数
# only the first day has a calculated "percent" value
def get_k_day_from_date(symbol, date, count=10):
    if symbol not in kday_cache:
        return None
    for (i, each) in enumerate(kday_cache[symbol]):
        if each['date'] == d2dt(date):
            rtn = kday_cache[symbol][i:(i+count)]
            if i > 0:
                rtn[0]['percent'] = (float(rtn[0]['close']) / float(kday_cache[symbol][i-1]['close']) - 1) * 100
            else:
                rtn[0]['percent'] = 0.0
            return rtn
    return None
    #result = db_ot.ths_kday.find({'symbol':symbol, 'date':{'$gte':d2dt(date)}}).sort('date')
    #return list(result)

#print(get_k_day_from_date("SH601139", datetime(2015, 1, 5, 0, 0)))

In [ ]:
timetag = "2012"
start_date = gen_time('2012-01-01 00:00:00').date()
end_date = gen_time('2012-12-31 00:00:00').date()
date = start_date
dates = []
while date <= end_date:
    if TradeCalendar.check_date(date):
        dates.append(date)
    date += timedelta(1)
    
# 对每个flag统计每个样本的表现
flag_stats = {} # {"...flag...":{"oneday":[], "fiveday":[], "onedayf":[], "fivedayf":[], ...}

# 根据日期查询并统计
import json
cur_year = 0
for date in dates:
    if date.year != cur_year:
        print(date.year)
        cur_year = date.year
    else:
        print('.', end='')
    f = open('iwencai_flags.json', 'r')
    try:
        flags = json.loads(f.read())
    except Exception as e:
        print(e)
        flags = []
    i = 0
    stop_symbols = {}
    for flag in flags:
        result = db_ot.iwencai_flag_history.find_one({'flag':flag, 'date':d2dt(date)})
        if not result:
            continue
        if flag not in flag_stats:
            flag_stats[flag] = {"oneday":[], "fiveday":[], "onedayf":[], "fivedayf":[]}
        for each_symbol in result['symbols']:
            sym = symbol_convert3(each_symbol)
            try:
                kday = get_k_day_from_date(sym, date)
            except ZeroDivisionError:
                continue
            if flag == '涨停':
                stop_symbols[sym] = True
            if not kday or len(kday) < 2:
                continue
            if kday[0]['date'] != d2dt(date):
                continue
            if float(kday[0]['close']) == 0.0:
                continue
            oneday = (float(kday[1]['close']) / float(kday[0]['close']) - 1) * 100
            if oneday >= 11.0: # kday data must have some bug, ignore this data
                continue
            flag_stats[flag]['oneday'].append(oneday)
            if len(kday) >= 6:
                fiveday = (float(kday[5]['close']) / float(kday[0]['close']) - 1) * 100
                flag_stats[flag]['fiveday'].append(fiveday)
                if kday[0]['percent'] <= 9.8 and sym not in stop_symbols:
                    flag_stats[flag]['fivedayf'].append(fiveday)
            if kday[0]['percent'] <= 9.8 and sym not in stop_symbols:
                flag_stats[flag]['onedayf'].append(oneday)
        i += 1
f = open('iwencai_flag_stats_'+timetag+'.json', 'w')
f.write(json.dumps(flag_stats))
f.close()

In [ ]:
# 计算和呈现不同flag的五日表现
result = []
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['fivedayf']:
        continue
    for percent in each['fivedayf']:
        avr += percent
    avr = avr / len(each['fivedayf'])
    result.append({'flag':flag, 'avr':avr, 'count':len(each['fivedayf'])})

sorted_x = sorted(result, key=lambda x:x['avr'])
for each in sorted_x:
    print("flag: %s, count: %d, mean: %f" % (each['flag'], each['count'], each['avr']))

In [ ]:
# 计算和呈现不同flag的单日表现
result = []
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['onedayf']:
        continue
    for percent in each['onedayf']:
        avr += percent
    avr = avr / len(each['onedayf'])
    result.append({'flag':flag, 'avr':avr, 'count':len(each['onedayf'])})

sorted_x = sorted(result, key=lambda x:x['avr'])
for each in sorted_x:
    print("flag: %s, count: %d, mean: %f" % (each['flag'], each['count'], each['avr']))

In [ ]:
# 将5日表现数据写入文件
fiveday_result = {}
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['fiveday']:
        continue
    for percent in each['fiveday']:
        avr += percent
    avr = avr / len(each['fiveday'])
    fiveday_result[flag] = {'avr':avr, 'count':len(each['fiveday'])}
f = open('iwencai_flag_performance_fiveday_%s.json' % timetag, 'w')
f.write(json.dumps(fiveday_result))
f.close()

fivedayf_result = {}
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['fivedayf']:
        continue
    for percent in each['fivedayf']:
        avr += percent
    avr = avr / len(each['fivedayf'])
    fivedayf_result[flag] = {'avr':avr, 'count':len(each['fivedayf'])}
f = open('iwencai_flag_performance_fivedayf_%s.json' % timetag, 'w')
f.write(json.dumps(fivedayf_result))
f.close()

# 将单日表现数据写入文件
oneday_result = {}
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['oneday']:
        continue
    for percent in each['oneday']:
        avr += percent
    avr = avr / len(each['oneday'])
    oneday_result[flag] = {'avr':avr, 'count':len(each['oneday'])}
f = open('iwencai_flag_performance_oneday_%s.json' % timetag, 'w')
f.write(json.dumps(oneday_result))
f.close()

onedayf_result = {}
for flag,each in flag_stats.items():
    avr = 0.0
    if not each['onedayf']:
        continue
    for percent in each['onedayf']:
        avr += percent
    avr = avr / len(each['onedayf'])
    onedayf_result[flag] = {'avr':avr, 'count':len(each['onedayf'])}
    
f = open('iwencai_flag_performance_onedayf_%s.json' % timetag, 'w')
f.write(json.dumps(onedayf_result))
f.close()